In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
import torch.nn as nn

In [2]:
from google.colab import files # 11mins
uploaded = files.upload()

Saving df1_test.csv to df1_test.csv
Saving df2_test.csv to df2_test.csv
Saving targets_test.csv to targets_test.csv
Saving trained_sample.pth to trained_sample.pth


In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 16)
        self.fc2 = nn.Linear(16, 9)
        self.fc3 = nn.Linear(9, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)



    def forward(self, x1, x2):
        # Element-wise subtraction
        x = x1 - x2
        # Forward pass with shared weights
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.fc4(x)
        if x.item() < 0.5:
            x = 0
        else:
            x = 1
        return x

    def loss(self, output, target):
        return self.criterion(output, target)

    def backward(self, loss):
        loss.backward()

    def train(self, x1, x2, target):
        # Forward pass
        output = self.forward(x1, x2)
        # Compute the loss
        loss = self.loss(output, target)
        # Zero the gradients
        self.optimizer.zero_grad()
        # Backward pass
        self.backward(loss)
        # Update the weights
        self.optimizer.step()
        return loss



In [32]:
# IF GPU is availible
# model = torch.load("trained_sample.pth")

#if no GPU
model = torch.load("trained_sample.pth", map_location=torch.device('cpu'))

# Data loading

In [33]:
input_x1 = pd.read_csv('df1_test.csv')
input_x2 = pd.read_csv('df2_test.csv')
target_y = pd.read_csv('targets_test.csv')

# Drop the first column
# input_x1 = input_x1.drop(input_x1.columns[0], axis=1)
# input_x2 = input_x2.drop(input_x2.columns[0], axis=1)
target_y = target_y.drop(target_y.columns[1], axis=1)

# Convert to numpy
input_x1 = input_x1.to_numpy()
input_x2 = input_x2.to_numpy()
target_y = target_y.to_numpy()

print(input_x1.shape)
print(input_x2.shape)
print(target_y.shape)



(5000, 20)
(5000, 20)
(5000, 1)


# Accuracy

In [42]:
correct = []
incorrect = []
accuracy = 0
total = 0
for i in range(len(input_x1)):
    x1 = torch.tensor(input_x1[i], dtype=torch.float32)
    x2 = torch.tensor(input_x2[i], dtype=torch.float32)
    y = target_y[i]
    output = model.forward(x1, x2)
    total += 1
    if output == y:
      accuracy += 1
      correct.append([x1, x2, output])
    else:
        incorrect.append([x1, x2, output])
print("Accuracy: ", accuracy/total)

Accuracy:  0.6786
